https://www.kaggle.com/datasets/shuvokumarbasak4004/global-energy-statistics?resource=download
This dataset provides comprehensive statistics on global energy production, consumption, and related metrics. It includes data on renewable energy, fossil fuels, electricity generation, and more. The data is collected from various reliable sources and is regularly updated to provide accurate insights into the global energy landscape.
Source: Kaggle

In [171]:
import os
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

1. Import and overview over the dataset:

In [172]:
import pandas as pd

df = pd.read_csv('Global Energy Statistics.csv')

# Overview
print('Head will be shown some lines of the data:')
print(df.head())
print('Info will be shown of the data:')
print(df.info())
print('Description will be shown of the data:')
print(df.describe())

Head will be shown some lines of the data:
         Country Energy Consumption  (BTU) World Share Per capita Yearly BTU
0          China       138,689,472,800,000      23.80%                98,342
1  United States        97,661,161,460,000      16.80%               296,130
2         Russia        32,832,743,720,000       5.60%               225,728
3          India        30,476,788,610,000       5.20%                22,505
4          Japan        19,402,928,820,000       3.30%               153,186
Info will be shown of the data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Country                    209 non-null    object
 1   Energy Consumption  (BTU)  209 non-null    object
 2   World Share                209 non-null    object
 3   Per capita Yearly BTU      209 non-null    object
dtypes: object(4)
memory usag

2. Edit the dataset

In [173]:
# Show missing values
print(df.isnull().sum())

# Delete missing values
df.dropna(inplace=True)

# Delete ,
df = df.replace(',', '', regex=True)

Country                      0
Energy Consumption  (BTU)    0
World Share                  0
Per capita Yearly BTU        0
dtype: int64


In the dataset are no field without a value. This unit is standard in the energ business.

Convert of the BTU uni(Unit for energy in USA) into kWh.
1 kWh = 3.412 BTU. Die Zahlen sollen umgerechnet werden in einer neuen Zeile dargestellt.

In [174]:
print("Spaltennamen im DataFrame:", df.columns)

# Bereinigen der Spaltennamen
df.columns = df.columns.str.strip()
print("Bereinigte Spaltennamen:", df.columns)

# Nicht-numerische Zeichen entfernen und in numerische Werte umwandeln
df['Energy Consumption  (BTU)'] = df['Energy Consumption  (BTU)'].str.replace(',', '').astype(float)
df['World Share'] = df['World Share'].str.replace('%', '').astype(float)
df['Per capita Yearly BTU'] = df['Per capita Yearly BTU'].str.replace(',', '').astype(float)

# Position der Spalte "World Share" finden
position = df.columns.get_loc('World Share')

# Werte in der Spalte "Energy Consumption  (BTU)" in kWh umrechnen
copyvalues = df['Energy Consumption  (BTU)'] / 3.412

# Wenn die Spalte "Energy Consumption (kWh)" bereits existiert, lösche sie
if 'Energy Consumption (kWh)' in df.columns:
    df.drop(columns=['Energy Consumption (kWh)'], inplace=True)

# Neue Spalte "Energy Consumption (kWh)" einfügen
df.insert(position, 'Energy Consumption (kWh)', copyvalues)

# DataFrame anzeigen
print(df)

Spaltennamen im DataFrame: Index(['Country', 'Energy Consumption  (BTU)', 'World Share',
       'Per capita Yearly BTU'],
      dtype='object')
Bereinigte Spaltennamen: Index(['Country', 'Energy Consumption  (BTU)', 'World Share',
       'Per capita Yearly BTU'],
      dtype='object')
                     Country  Energy Consumption  (BTU)  \
0                      China               1.386895e+14   
1              United States               9.766116e+13   
2                     Russia               3.283274e+13   
3                      India               3.047679e+13   
4                      Japan               1.940293e+13   
..                       ...                        ...   
204  Saint Pierre & Miquelon               1.982000e+09   
205                    Tonga               1.807000e+09   
206             Cook Islands               1.677278e+09   
207         Falkland Islands               8.290000e+07   
208                 Kiribati               4.610000e+07   

     

Groups and analyze the dataset:
1. Group app industrialized countries into one group and sum up energy consumption
   United States, Germany, Japan, France, United Kingdom, Canada, South Korea, Italy
2. Sort all countries: Per capita Yearly (BTU) from high to low.

In [180]:
# part 1 group
selected_countries = ['China','United States', 'Germany', 'Japan', 'France', 'United Kingdom', 'Canada', 'South Korea', 'Italy']
industry_df = df[df['Country'].isin(selected_countries)]
print(industry_df)

total_energy_consumption_industry = industry_df['Energy Consumption  (BTU)'].sum()
total_world_share_industry = industry_df['World Share'].sum()
print("Die Totale Energie der Industrielaender betraegt:")
print(total_energy_consumption_industry)
print("Die Industrielaender verbrauchen soviele Prozent der gesamten Energie in Prozent:")
print(total_world_share_industry)

           Country  Energy Consumption  (BTU)  Energy Consumption (kWh)  \
0            China               1.386895e+14              4.064756e+13   
1    United States               9.766116e+13              2.862285e+13   
4            Japan               1.940293e+13              5.686673e+12   
5           Canada               1.506212e+13              4.414456e+12   
6          Germany               1.406254e+13              4.121495e+12   
7      South Korea               1.265909e+13              3.710166e+12   
11          France               1.024166e+13              3.001660e+12   
12  United Kingdom               8.190850e+12              2.400601e+12   
15           Italy               6.771608e+12              1.984645e+12   

    World Share  Per capita Yearly BTU  
0         23.80                98342.0  
1         16.80               296130.0  
4          3.30               153186.0  
5          2.60               412047.0  
6          2.40               170198.0  
7  

We need to import the data, but first there are several files that we need to work. The data is separated in terms of decade. Therefore, we write a script that looks for all the files containing the data, read and combine them together into a single dataframe that we can work on.

In [4]:
# Get the current working directory
cwd = os.getcwd()

In [9]:
files = os.listdir(cwd)

dfs = []

for file in files:
    
    if file.endswith('.csv'):
        
        df = pd.read_csv(file)
        dfs.append(df)
        
df = pd.concat(dfs, ignore_index = True)
df.shape

(667, 14)

### Data overview 

Let us have an overview of the dataset

In [10]:
df.head(4)

,title,artist,top_genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,No Scrubs,TLC,atl hip hop,1999,93,68,74,-4,7,59,214,3,10,79
1,I Want It That Way,Backstreet Boys,boy band,2001,99,70,69,-6,16,48,214,22,3,76
2,It Wasn't Me,Shaggy,dance pop,2000,95,61,85,-5,31,65,228,6,7,76
3,"Because You Loved Me (Theme from ""Up Close and...",CÃ©line Dion,canadian pop,1996,120,47,61,-10,10,18,274,31,3,74


In [11]:
df.columns

Index(['title', 'artist', 'top_genre', 'year', 'bpm', 'nrgy', 'dnce', 'dB',
       'live', 'val', 'dur', 'acous', 'spch', 'pop'],
      dtype='object')

1. top genre: genre of the song

2. year: year of the song (due to re-releases, the year might not correspond to the release year of the original song)

3. bpm(beats per minute): beats per minute

4. nrgy(energy): energy of a song, the higher the value the more energetic the song is

5. dnce(danceability): the higher the value, the easier it is to dance to this song.

6. dB(loudness): the higher the value, the louder the song.

7. live(liveness): the higher the value, the more likely the song is a live recording.

8. val(valence): the higher the value, the more positive mood for the song.

9. dur(duration): the duration of the song.

10. acous(acousticness): the higher the value the more acoustic the song is.

11. spch(speechiness): the higher the value the more spoken word the song contains.

12. pop(popularity): the higher the value the more popular the song is.

##### Business Question 

Now let us formulate a business question for this project

1. What was the most popular genre in each decade?
2. Who was the most listened to artist in each decade?
2. What was the average danceability score of most recommended songs in every decade? 
3. What was the average bpm score of most recommended songs in every decade? 
4. What is the secret behind a popular song on online streaming platforms?

### Data Cleaning 

Let us drop unnecessary columns

In [12]:
df = df.drop(['year', 'title'], axis = 1)
df.head(4)

,artist,top_genre,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,TLC,atl hip hop,93,68,74,-4,7,59,214,3,10,79
1,Backstreet Boys,boy band,99,70,69,-6,16,48,214,22,3,76
2,Shaggy,dance pop,95,61,85,-5,31,65,228,6,7,76
3,CÃ©line Dion,canadian pop,120,47,61,-10,10,18,274,31,3,74


Let us check for missing datapoints

In [13]:
attributes = ['artist', 'top_genre', 'bpm', 'nrgy', 'dnce', 'dB',
       'live', 'val', 'dur', 'acous', 'spch', 'pop']

In [ ]:
for attribute in attributes:
    
    null_check = pd.isnull(df[attribute])
    print(df[null_check])

We can see that the 'top_genre' column has several missing values. We can drop them

<div class="alert alert-block alert-warning">

<b>!! WARNING !!</b> This is a small dataset and dropping some can hugely compromise the integrity of the analytical results.

</div>

So we are gping to use a different technique to fill the missing values. We fill any missing values with the most popular value in the corresponding column

Let us iterate through the columns, find the common value and then fill it where there is any missing value.

In [15]:
for column in df.columns:
    
    mode_value = df[column].mode()[0]
    df[column].fillna(mode_value, inplace = True)

Let us confirm that all the missing values have been replaced

In [16]:
attributes = ['top_genre']

In [17]:
for attribute in attributes:
    
    null_check = pd.isnull(df[attribute])
    print(df[null_check])

Empty DataFrame
Columns: [artist, top_genre, bpm, nrgy, dnce, dB, live, val, dur, acous, spch, pop]
Index: []


We do not have any missing values hence the loop worked successfully. So let us answer the business questions.

###### 1. What was the most popular genre in each decade?

In [23]:
dfs = []
years = ['1950', '1960', '1970', '1980', '1990', '2000', '2010']

for file in files:
    
    if file.endswith('.csv'):
        
        df = pd.read_csv(file)
        df['decade'] = ''
        
        for i in range(len(df)):
            
            for year in years:
                
                if year in file:
                    
                    df['decade'].loc[i] = year
                    
        dfs.append(df)
df = pd.concat(dfs, ignore_index = True)
df.head(4)

,title,artist,top_genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,decade
0,No Scrubs,TLC,atl hip hop,1999,93,68,74,-4,7,59,214,3,10,79,1990
1,I Want It That Way,Backstreet Boys,boy band,2001,99,70,69,-6,16,48,214,22,3,76,1990
2,It Wasn't Me,Shaggy,dance pop,2000,95,61,85,-5,31,65,228,6,7,76,1990
3,"Because You Loved Me (Theme from ""Up Close and...",CÃ©line Dion,canadian pop,1996,120,47,61,-10,10,18,274,31,3,74,1990


Let us confirm if the decade is an integer or a string using the following loop.

In [ ]:
for i in range(len(df)):
    
    print(type(df['decade'].loc[i]))

Let us convert the string values to integers using the code below

In [25]:
df['decade'] = df['decade'].astype(int)

Let us confirm if the decades are now integers

In [ ]:
for i in range(len(df)):
    
    print(type(df['decade'].loc[i]))

Let us check the available genres in our dataset

In [ ]:
df['top_genre'].unique()

In [30]:
df.groupby('decade').top_genre.agg(pd.Series.mode).reset_index(name = 'Genre')

,decade,Genre
0,1950,adult standards
1,1960,adult standards
2,1970,album rock
3,1980,album rock
4,1990,dance pop
5,2000,dance pop
6,2010,dance pop


##### 2. Who was the most listened to artist in each decade? 

In [31]:
df.groupby('decade').artist.agg(pd.Series.mode).reset_index(name = 'Artist')

,decade,Artist
0,1950,"[Craig Douglas, Dean Martin, The Fleetwoods]"
1,1960,The Beatles
2,1970,"[Elton John, Queen]"
3,1980,Madonna
4,1990,Mariah Carey
5,2000,Rihanna
6,2010,Post Malone


##### 3. What was the average danceability score of most recommended songs in every decade? 

In [32]:
df.groupby('decade').dnce.agg(['count', 'mean'])

,count,mean
decade,,
1950,73,51.027397
1960,97,51.175258
1970,104,56.596154
1980,105,62.276190
1990,88,65.147727
2000,100,68.530000
2010,100,65.370000
